# Avaliação de Desempenho em Modelos de Machine Learning

Este notebook traz um panorama sobre **avaliação de desempenho** em modelos de *Machine Learning*, cobrindo:

1. **Modelos de classificação**: principais métricas (acurácia, precisão, revocação, F1-score), como interpretar e para que cenários são mais apropriadas.
2. **Modelos de regressão**: principais métricas (MSE, RMSE, MAPE), como interpretar e para que cenários são mais apropriadas.
3. **Modelos não supervisionados** (ex.: clustering): métricas como Silhouette Score.
4. **Validação cruzada** (Cross-Validation): técnica de particionamento dos dados em múltiplos subconjuntos para medir a capacidade de generalização dos modelos.

---
## 1. O que é Avaliação de Desempenho?

Após o treinamento de um modelo de machine learning, precisamos avaliar quão bem ele consegue **generalizar** para dados nunca vistos. Para isso, utilizamos:

- **Um conjunto de teste**: dados não usados durante o treinamento.
- **Métricas apropriadas**: valores quantitativos que representam o desempenho do modelo.

Se o modelo é supervisionado (tem uma variável alvo conhecida), dividimos em **classificação** ou **regressão**:
- **Classificação**: prever classes (ex.: spam ou não spam). 
- **Regressão**: prever valores contínuos (ex.: preço de imóveis).

Se o modelo é não supervisionado, por exemplo de **clustering** (agrupamento), precisamos de outras métricas que avaliem a coesão e separação dos clusters.


## 2. Preparação: Bibliotecas e Conjuntos de Dados

Vamos começar importando as bibliotecas necessárias e criando (ou carregando) conjuntos de dados para exemplificarmos as métricas.


In [22]:
%pip install numpy pandas matplotlib seaborn scikit-learn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_classification, make_regression, make_blobs
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    mean_squared_error, mean_absolute_percentage_error,
    silhouette_score
)
from sklearn.preprocessing import StandardScaler
from math import sqrt
from sklearn.cluster import KMeans

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)
np.random.seed(42)

print("Bibliotecas importadas!")

Note: you may need to restart the kernel to use updated packages.
Bibliotecas importadas!


### 2.1. Conjunto de dados para Classificação

Vamos criar um dataset sintético de classificação binária (classes 0 e 1).

In [23]:
# Criando dados de classificação binária com 2 features
X_class, y_class = make_classification(
    n_samples=1000,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_clusters_per_class=1,
    random_state=42
)

# Separando em treino e teste
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_class, y_class,
    test_size=0.3,
    random_state=42
)

print("Formato dados de Classificação - Treino:", X_train_cls.shape)
print("Formato dados de Classificação - Teste :", X_test_cls.shape)

Formato dados de Classificação - Treino: (700, 2)
Formato dados de Classificação - Teste : (300, 2)


### 2.2. Conjunto de dados para Regressão

Vamos criar um dataset sintético de regressão, onde nossa **variável alvo (y)** é contínua.

In [24]:
# Criando dados de regressão com 1 feature
X_reg, y_reg = make_regression(
    n_samples=1000,
    n_features=1,
    noise=20,
    random_state=42
)

y_reg = y_reg + 100  # Adiciona um deslocamento para evitar valores próximos de zero

# Separando em treino e teste
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg,
    test_size=0.3,
    random_state=42
)

print("Formato dados de Regressão - Treino:", X_train_reg.shape)
print("Formato dados de Regressão - Teste :", X_test_reg.shape)

Formato dados de Regressão - Treino: (700, 1)
Formato dados de Regressão - Teste : (300, 1)


### 2.3. Conjunto de dados para Clustering (Não Supervisionado)

Como exemplo de modelo não supervisionado, usaremos um dataset de *blobs* (grupos) gerado com `make_blobs`, para avaliarmos com **Silhouette Score**.

In [25]:
# Criando dados de clustering com 3 centros
X_clust, _ = make_blobs(
    n_samples=300,
    n_features=2,
    centers=3,
    cluster_std=1.0,
    random_state=42
)
print("Formato dados de Clustering:", X_clust.shape)

Formato dados de Clustering: (300, 2)


---
## 3. Modelos Supervisionados - Classificação

Um problema de classificação prediz uma **classe** (categorias, rótulos) como saída. Por exemplo: *spam ou não spam*, *positivo ou negativo*, *fraude ou não fraude*.

### 3.1 Treinando um modelo de classificação (Ex.: Regressão Logística)

Vamos utilizar a **Regressão Logística** apenas como exemplo. A ideia aqui é ilustrar como calcular as principais métricas.


In [26]:
# Treinando um modelo de classificação
clf = LogisticRegression()
clf.fit(X_train_cls, y_train_cls)

# Previsões no conjunto de teste
y_pred_cls = clf.predict(X_test_cls)

print("Modelo de classificação treinado!")

Modelo de classificação treinado!


### 3.2 Principais Métricas de Classificação

**Acurácia**: Proporção de acertos do modelo. Calcula quantas vezes o modelo previu corretamente em relação ao total de previsões.
- Sentido: boa métrica quando há *classes balanceadas* e não é crítico diferenciar classes positivas e negativas.

**Precisão (Precision)**: Entre as instâncias preditas como positivas, quantas realmente são positivas?
- Sentido: útil quando o **custo de um falso positivo** é alto (ex.: diagnóstico médico, detecção de fraude).

**Revocação (Recall)**: Entre as instâncias que são de fato positivas, quantas o modelo conseguiu identificar?
- Sentido: útil quando o **custo de um falso negativo** é alto (ex.: deixar de detectar um caso positivo).

**F1-Score**: Média harmônica entre Precisão e Revocação.
- Sentido: equilibrar Precisão e Revocação em um número único, útil quando as classes estão desbalanceadas.


### 3.3 Exemplo de Cálculo das Métricas de Classificação
Vamos calcular cada métrica a partir das previsões `y_pred_cls`.

In [27]:
acc = accuracy_score(y_test_cls, y_pred_cls)
prec = precision_score(y_test_cls, y_pred_cls)
rec = recall_score(y_test_cls, y_pred_cls)
f1 = f1_score(y_test_cls, y_pred_cls)

print(f"Acurácia : {acc:.4f}")
print(f"Precisão : {prec:.4f}")
print(f"Revocação: {rec:.4f}")
print(f"F1-score : {f1:.4f}")

Acurácia : 0.8833
Precisão : 0.9275
Revocação: 0.8366
F1-score : 0.8797


---
## 4. Modelos Supervisionados - Regressão

Um problema de regressão prevê **valores contínuos** como saída. Por exemplo, *preço de imóveis, temperatura, vendas*, etc.

### 4.1 Treinando um modelo de Regressão (Ex.: Regressão Linear)


In [28]:
# Treinando um modelo de Regressão Linear
reg = LinearRegression()
reg.fit(X_train_reg, y_train_reg)

# Previsões no conjunto de teste
y_pred_reg = reg.predict(X_test_reg)

print("Modelo de regressão treinado!")

Modelo de regressão treinado!


### 4.2 Principais Métricas de Regressão

**MSE (Mean Squared Error)**: Erro médio quadrático. Faz a média dos quadrados dos erros (diferença entre valor real e valor previsto).
- Sentido: penaliza erros grandes de forma mais forte (por serem quadrados).

**RMSE (Root Mean Squared Error)**: Raiz quadrada do MSE. Traz o erro de volta à mesma escala da variável alvo.
- Cálculo manual: \( RMSE = \sqrt{MSE} \)

**MAPE (Mean Absolute Percentage Error)**: Erro percentual médio absoluto. Mede o erro em porcentagem em relação aos valores reais.
- Sentido: útil para interpretar o erro como % dos valores reais.
- Cuidado: pode não ser confiável quando há valores reais próximos de zero.


### 4.3 Exemplo de Cálculo das Métricas de Regressão
- Veremos o cálculo manual para MSE e RMSE, e o uso de funções do `scikit-learn`.


In [29]:
# Cálculo manual MSE e RMSE
errors = y_test_reg - y_pred_reg
mse_manual = np.mean(errors**2)
rmse_manual = np.sqrt(mse_manual)

# Usando scikit-learn
mse_sklearn = mean_squared_error(y_test_reg, y_pred_reg)
rmse_sklearn = np.sqrt(mse_sklearn)

# MAPE (scikit-learn >= 1.0), se versão antiga, precisa fazer manual.
mape_value = mean_absolute_percentage_error(y_test_reg, y_pred_reg)

print("--- Cálculo Manual ---")
print(f"MSE  (manual): {mse_manual:.4f}")
print(f"RMSE (manual): {rmse_manual:.4f}")

print("\n--- scikit-learn ---")
print(f"MSE  (sklearn): {mse_sklearn:.4f}")
print(f"RMSE (sklearn): {rmse_sklearn:.4f}")
print(f"MAPE (sklearn): {mape_value*100:.2f}%")

--- Cálculo Manual ---
MSE  (manual): 417.5302
RMSE (manual): 20.4336

--- scikit-learn ---
MSE  (sklearn): 417.5302
RMSE (sklearn): 20.4336
MAPE (sklearn): 18.72%


### Importante
O cálculo do MAPE pode ser problemático em datasets com valores reais (`y`) muito próximos de zero. Isso ocorre porque o erro relativo será inflacionado, resultando em um MAPE elevado e não representativo.

Para evitar isso:
1. Garanta que os valores reais estejam suficientemente distantes de zero.
2. Use métricas alternativas, como **SMAPE** ou **MAE**, em casos onde os valores de `y` possam ser muito pequenos.
3. No exemplo a seguir, deslocamos os valores de `y` para evitar problemas no cálculo do MAPE.



---
## 5. Modelos Não Supervisionados - Clustering

Em problemas não supervisionados de *clustering*, não existe uma variável alvo conhecida. Ainda assim, podemos usar métricas que avaliam a coesão e separação dos grupos formados.

**Silhouette Score**: varia em geral de -1 a 1. 
- Valores próximos de 1 indicam que os pontos estão **bem separados** de outros clusters e **próximos** dos pontos do próprio cluster.
- Valores próximos de 0 indicam sobreposição ou fronteiras difusas entre clusters.
- Valores negativos indicam que os pontos podem estar em clusters "errados" (mais próximos de outro cluster do que do seu próprio).


In [30]:
# Aplicando KMeans em dados de clustering sintéticos
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(X_clust)

# Calculando o Silhouette Score
score = silhouette_score(X_clust, labels)
print(f"Silhouette Score: {score:.4f}")

Silhouette Score: 0.8480


Como estamos usando `make_blobs` com 3 centros, esperamos encontrar um *Silhouette Score* razoavelmente alto. Em problemas reais, essa métrica pode ajudar a escolher o número de clusters ou comparar diferentes algoritmos de clustering.

---
## 6. Validação Cruzada (Cross-Validation)

A **Validação Cruzada** é uma técnica que divide o conjunto de dados em vários blocos ("folds"). Em cada iteração, um fold diferente é usado como conjunto de teste, enquanto os demais folds servem para o treinamento.

### Vantagens:
- Avaliação mais **robusta**, pois usamos diferentes partições para treino e teste.
- **Reduz a variância** das estimativas de desempenho.

Vamos demonstrar a validação cruzada para um modelo de classificação usando **acurácia** como métrica.


In [31]:
# Exemplo de Cross-Validation em Classificação
scores = cross_val_score(
    LogisticRegression(),
    X_class,
    y_class,
    cv=5,               # 5 folds
    scoring='accuracy'  # Métrica: acurácia
)

print(f"Scores em cada fold: {scores}")
print(f"Acurácia média   : {scores.mean():.4f}")
print(f"Desvio padrão     : {scores.std():.4f}")

Scores em cada fold: [0.875 0.91  0.89  0.925 0.915]
Acurácia média   : 0.9030
Desvio padrão     : 0.0181


Com isso, obtemos o desempenho do modelo em 5 partições diferentes. A **média** indica a performance geral, enquanto o **desvio padrão** mostra a variação de desempenho.

## Conclusão

1. **Classificação**: acurácia, precisão, revocação e F1-score são fundamentais para compreender o desempenho em cenários diferentes (classes balanceadas, custo de erros).
2. **Regressão**: MSE, RMSE e MAPE são métricas clássicas para entender o quão distantes as previsões estão dos valores reais.
3. **Não supervisionado**: métricas como *Silhouette Score* são úteis para mensurar a qualidade dos clusters.
4. **Validação cruzada**: técnica essencial para ter uma visão mais confiável do poder de generalização do modelo.